## Langchain (prompt template)

Um prompt template é uma estrutura reutilizável que define como os inputs do usuário serão organizados e formatados antes de serem enviados ao modelo de linguagem. Ele funciona como um "molde" para gerar prompts consistentes, dinâmicos e mais fáceis de manter.

### 1. Bibliotecas

In [1]:
from langchain_openai.llms import OpenAI
import os
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [2]:
# Acessando API

load_dotenv()
secret_key = os.getenv("OPEN_API_KEY")

In [3]:
# Selecionando o modelo

llm = OpenAI(model='gpt-3.5-turbo-instruct',
            openai_api_key=secret_key)

### 2. Prompt template padrão/básico

In [4]:
# Prompt template 1

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário:
{pergunta}
""")

In [5]:
print(prompt_template.format(pergunta='O que é um SaaS?'))


Responda a seguinte pergunta do usuário:
O que é um SaaS?



### 3. Prompt template com limitação de saída

In [6]:
# Prompt template 2

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
""")

In [7]:
print(prompt_template.format(pergunta='O que é um SaaS?',n_palavras=15))


Responda a seguinte pergunta do usuário em até 15 palavras:
O que é um SaaS?



Esse prompt template abaixo é uma variação do anterior, mas com um diferencial importante: ele usa partial_variables para pré-definir um dos parâmetros (n_palavras = 10). Usar partial_variables no PromptTemplate é como definir um valor default para uma ou mais variáveis do prompt.

In [8]:
# Prompt template 3

prompt_template = PromptTemplate.from_template("""
Responda a seguinte pergunta do usuário em até {n_palavras} palavras:
{pergunta}
""", partial_variables={'n_palavras':10})

In [9]:
print(prompt_template.format(pergunta='O que é Langchain?'))


Responda a seguinte pergunta do usuário em até 10 palavras:
O que é Langchain?



### 4. Prompt template com múltiplos prompts

Esse código demonstra como criar vários blocos de prompt reutilizáveis (templates parciais), cada um responsável por uma instrução específica,como limitar palavras, linhas ou definir o idioma da resposta. Depois, esses blocos são concatenados para formar um prompt final completo e parametrizado.

In [10]:
# Prompt template 4 (múltiplos prompts)

template_word_count = PromptTemplate.from_template("""
Responda a pergunta em até {n_palavras} palavras.
""")

template_line_count = PromptTemplate.from_template("""
Responda a pergunta em até {n_linhas} linhas.
""")

template_idioma = PromptTemplate.from_template("""
Retorne a resposta no {idioma}.
""")

template_final = (template_word_count + template_line_count + template_idioma + 'Responda a pergunta seguindo as instruções {pergunta}')

In [11]:
print(template_final)

input_variables=['idioma', 'n_linhas', 'n_palavras', 'pergunta'] template='\nResponda a pergunta em até {n_palavras} palavras.\n\nResponda a pergunta em até {n_linhas} linhas.\n\nRetorne a resposta no {idioma}.\nResponda a pergunta seguindo as instruções {pergunta}'


In [12]:
prompt_final = template_final.format(n_palavras=15, n_linhas=1, idioma='inglês', pergunta='O que é o sol?')
llm.invoke(prompt_final)

'\nWhat is the sun?'

### 5. Prompt template com prompts encadeados com chat

O código abaixo usa ChatPromptTemplate para criar uma conversa estruturada com múltiplas mensagens, separando os papéis de system, human e ai. Ele define o comportamento do assistente, insere mensagens anteriores e recebe uma nova pergunta do usuário, com variáveis como {nome_assistente} e {pergunta} preenchidas dinamicamente. Isso permite simular um chat com contexto e gerar respostas mais naturais.

In [13]:
chat_template = ChatPromptTemplate.from_template('Essa é minha dúvida: {duvida}')
chat_template.format_messages(duvida='Quem é você?')

[HumanMessage(content='Essa é minha dúvida: Quem é você?')]

In [14]:
chat_template = ChatPromptTemplate.from_messages([
    ('system','Você é um assistente irônico e se chama {nome_assistente}'),
    ('human','Olá, como vai?'),
    ('ai','Estou bem, como posso ajudar?'),
    ('human','{pergunta}')
])

In [15]:
chat_template

ChatPromptTemplate(input_variables=['nome_assistente', 'pergunta'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['nome_assistente'], template='Você é um assistente irônico e se chama {nome_assistente}')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Olá, como vai?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Estou bem, como posso ajudar?')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pergunta'], template='{pergunta}'))])

In [16]:
chat_template.format_messages(nome_assistente='Botx', pergunta='Qual o seu nome?')

[SystemMessage(content='Você é um assistente irônico e se chama Botx'),
 HumanMessage(content='Olá, como vai?'),
 AIMessage(content='Estou bem, como posso ajudar?'),
 HumanMessage(content='Qual o seu nome?')]

In [17]:
chat = ChatOpenAI(api_key=secret_key)

In [18]:
chat.invoke(chat_template.format_messages(nome_assistente='BotX',pergunta='Qual o seu nome?'))

AIMessage(content='Meu nome é BotX, o assistente irônico. Como posso te ajudar hoje?', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 54, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b905589c-88e8-4901-a77f-6ca5a626fa61-0')

### 6. Prompt-few-shot

O código demonstra o uso de FewShotPromptTemplate no LangChain, onde exemplos anteriores de perguntas e respostas são usados como referência para ensinar ao modelo um padrão de raciocínio passo a passo. Cada exemplo mostra como decompor perguntas complexas em perguntas intermediárias, fornecendo uma estrutura para chegar à resposta final. O modelo então aplica esse mesmo estilo ao responder uma nova pergunta (input='O que Darwin estudou?'), aprendendo a partir dos exemplos fornecidos. Isso é útil para melhorar a qualidade e consistência das respostas, especialmente em tarefas que exigem raciocínio ou decomposição de problemas.

In [19]:
exemplos = [
    {"pergunta": "Qual é a maior montanha do mundo, o Monte Everest ou o K2?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 
""", 
    }, 
    {"pergunta": "Quem nasceu primeiro, Charles Darwin ou Albert Einstein?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 
""", 
    }, 
    {"pergunta": "Quem foi o pai de Napoleão Bonaparte?",
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte? 
Resposta intermediária: Napoleão Bonaparte foi um líder militar e imperador francês. 
Pergunta de acompanhamento: Quem foi o pai de Napoleão Bonaparte? 
Resposta intermediária: O pai de Napoleão Bonaparte foi Carlo Buonaparte. 
Então a resposta final é: Carlo Buonaparte 
""", 
    },
    {"pergunta": "Os filmes 'O Senhor dos Anéis' e 'O Hobbit' foram dirigidos pelo mesmo diretor?", 
     "resposta": 
     """São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem dirigiu 'O Senhor dos Anéis'? 
Resposta intermediária: 'O Senhor dos Anéis' foi dirigido por Peter Jackson. 
Pergunta de acompanhamento: Quem dirigiu 'O Hobbit'? 
Resposta intermediária: 'O Hobbit' também foi dirigido por Peter Jackson. 
Então a resposta final é: Sim 
""",
    },
]

In [20]:
example_prompt = PromptTemplate(
    input_variables=["pergunta", "resposta"],
    template="Pergunta {pergunta}\n{resposta}"
)

example_prompt.format(**exemplos[0])

'Pergunta Qual é a maior montanha do mundo, o Monte Everest ou o K2?\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Qual é a altura do Monte Everest? \nResposta intermediária: O Monte Everest tem 8.848 metros de altura. \nPergunta de acompanhamento: Qual é a altura do K2? \nResposta intermediária: O K2 tem 8.611 metros de altura. \nEntão a resposta final é: Monte Everest \n'

In [21]:
prompt = FewShotPromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt,
    suffix="Pergunta: {input}",
    input_variables=["input"]
)

In [22]:
print(prompt.format(input='Quem nasceu primeiro, Nikola Tesla ou Albert Einstein?'))

Pergunta Qual é a maior montanha do mundo, o Monte Everest ou o K2?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Qual é a altura do Monte Everest? 
Resposta intermediária: O Monte Everest tem 8.848 metros de altura. 
Pergunta de acompanhamento: Qual é a altura do K2? 
Resposta intermediária: O K2 tem 8.611 metros de altura. 
Então a resposta final é: Monte Everest 


Pergunta Quem nasceu primeiro, Charles Darwin ou Albert Einstein?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quando nasceu Charles Darwin? 
Resposta intermediária: Charles Darwin nasceu em 12 de fevereiro de 1809. 
Pergunta de acompanhamento: Quando nasceu Albert Einstein? 
Resposta intermediária: Albert Einstein nasceu em 14 de março de 1879. 
Então a resposta final é: Charles Darwin 


Pergunta Quem foi o pai de Napoleão Bonaparte?
São necessárias perguntas de acompanhamento aqui: Sim. 
Pergunta de acompanhamento: Quem foi Napoleão Bonaparte?

In [23]:
llm.invoke(prompt.format(input='Quem nasceu primeiro, Nikola Tesla ou Albert Einstein?'))

'\nSão necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quando nasceu Nikola Tesla? \nResposta intermediária: Nikola Tesla nasceu em 10 de julho de 1856. \nPergunta de acompanhamento: Quando nasceu Albert Einstein? \nResposta intermediária: Albert Einstein nasceu em 14 de março de 1879. \nEntão a resposta final é: Nikola Tesla'

### 7. Prompt-few-shot em chat

O código abaixo usa o FewShotChatMessagePromptTemplate para criar uma interação de chat com base em exemplos de perguntas e respostas anteriores. Ele define um template de chat com mensagens do tipo human e ai, que servem como exemplos para ensinar o modelo a responder perguntas similares. Após criar o template de exemplo, ele é usado dentro de um ChatPromptTemplate para construir a conversa final, onde uma nova pergunta ("Qual ginasta feminina no Brasil ganhou mais medalhas?") é formatada e enviada ao modelo. Esse processo permite ao modelo entender o padrão de respostas esperadas com base em exemplos dados, melhorando a consistência e a qualidade das respostas para perguntas semelhantes.

In [24]:
chat = ChatOpenAI(api_key=secret_key)

In [25]:
example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{pergunta}"),
     ("ai", "{resposta}")]
)

print(example_prompt.format_messages(**exemplos[0]))

[HumanMessage(content='Qual é a maior montanha do mundo, o Monte Everest ou o K2?'), AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Qual é a altura do Monte Everest? \nResposta intermediária: O Monte Everest tem 8.848 metros de altura. \nPergunta de acompanhamento: Qual é a altura do K2? \nResposta intermediária: O K2 tem 8.611 metros de altura. \nEntão a resposta final é: Monte Everest \n')]


In [26]:
few_shot_template = FewShotChatMessagePromptTemplate(
    examples=exemplos,
    example_prompt=example_prompt
)

prompt_final = ChatPromptTemplate.from_messages([
    few_shot_template,
    ("human", "{input}")
])

prompt = prompt_final.format_messages(input="Rebeca Andrade ganhou mais medalhas que Daiane dos Santos?")
# few_shot_template.format_messages()

In [27]:
chat.invoke(prompt)

AIMessage(content='São necessárias perguntas de acompanhamento aqui: Sim. \nPergunta de acompanhamento: Quantas medalhas conquistou Rebeca Andrade? \nResposta intermediária: Rebeca Andrade conquistou duas medalhas nas Olimpíadas de Tóquio 2020, sendo uma prata e um ouro. \nPergunta de acompanhamento: Quantas medalhas conquistou Daiane dos Santos? \nResposta intermediária: Daiane dos Santos conquistou uma medalha de ouro nos Jogos Pan-Americanos de 2003 e uma medalha de ouro nos Jogos Sul-Americanos de 2002. \nEntão a resposta final é: Não, Daiane dos Santos conquistou mais medalhas do que Rebeca Andrade.', response_metadata={'token_usage': {'completion_tokens': 172, 'prompt_tokens': 523, 'total_tokens': 695, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 